# Import

In [1]:
import json
from datetime import datetime, timedelta
from tkinter import *
from tkinter import ttk, filedialog, messagebox
import os
from collections import Counter, defaultdict

# Data Reader/Writter Functions
```python
#Read json
load("file_path.txt")
#Convert str to json
str_to_json(data)
#Write json to txt
write(data, "file_path.txt")
```

In [2]:
def load(filepath):
    with open(filepath, 'r', encoding='cp037') as f:
        return json.load(f)

def write(data, filepath):
    json_str = json.dumps(data, ensure_ascii=False, indent=2)
    with open(filepath, 'w', encoding='cp037') as f:
        f.write(json_str)

def str_to_json(s):
    return eval(s)

# Data Reader

In [3]:
class datasets:
    def __init__(self):
        self.users = load('datasets/users.txt')
        self.menu = load('datasets/menu.txt')
        self.ingredients = load('datasets/ingredients.txt')
        self.ingredient_order = load('datasets/ingredient_order.txt')
        self.orders = load('datasets/orders.txt')
        self.finance = load('datasets/finance.txt')
        self.feedbacks = load('datasets/feedbacks.txt')
        self.threshold = load('datasets/threshold.txt')

    def get_users(self):
        return self.users

    def get_menu(self):
        return self.menu

    def get_iventory(self):
        return self.ingredients

    def get_ingredient_order(self):
        return self.ingredient_order

    def get_orders(self):
        return self.orders

    def get_finance(self):
        return self.finance

    def get_feedback(self):
        return self.feedbacks

    def get_threshold(self):
        return self.threshold

# Linear Regression

ŷ / y-hat stands for prediction

Linear Equation: ŷ = ax + b

Loss (MSE Loss) = (y - ŷ)^2

Expansion of Loss: L = y^2 - 2y(ax + b) + (ax + b)^2

                = y^2 - 2ayx - 2yb + a^2x^2 + 2axb + b^2
Derivative of a: dL/da = -2yx + 2ax^2 + 2xb

Derivative of b: dL/db = -2y + 2ax + 2b

New Loss(a) = a - learning rate(lr) * da

New Loss(b) = a - learning rate(lr) * db

In [4]:
class Model:
    def __init__(self):
        with open('augmented_orders.json', 'r', encoding='utf-8') as f:
            self.orders_data = json.load(f)

        self.ingredients_data = load('datasets/ingredients.txt')

        completed_orders = [o for o in self.orders_data['orders'] if o['status'] == 'Completed']
        order_time_map = {o['order_id']: o['order_time'] for o in completed_orders}

        daily_dish_sales = {}
        for item in self.orders_data['order_items']:
            oid = item['order_id']
            if oid not in order_time_map:
                continue
            date_str = order_time_map[oid][:10]
            dish_id = item['dish_id']
            qty = item['quantity']
            key = (date_str, dish_id)
            daily_dish_sales[key] = daily_dish_sales.get(key, 0) + qty

        recipe_dict = {}
        for r in self.ingredients_data['recipes']:
            recipe_dict[r['dish_id']] = [(ing['ingredient_id'], ing['quantity']) for ing in r['ingredients']]

        daily_ingredient_sales = {}
        for (date_str, dish_id), dish_qty in daily_dish_sales.items():
            if dish_id not in recipe_dict:
                continue
            for ing_id, ing_qty_per_dish in recipe_dict[dish_id]:
                key = (date_str, ing_id)
                daily_ingredient_sales[key] = daily_ingredient_sales.get(key, 0) + dish_qty * ing_qty_per_dish

        dates = sorted(list(set(date for date, _ in daily_ingredient_sales.keys())))
        date_to_num = {date: i for i, date in enumerate(dates)}
        ingredient_ids = list(set(ing_id for _, ing_id in daily_ingredient_sales.keys()))

        ingredient_time_series = {ing_id: [0.0] * len(dates) for ing_id in ingredient_ids}
        for (date, ing_id), qty in daily_ingredient_sales.items():
            idx = date_to_num[date]
            ingredient_time_series[ing_id][idx] = qty

        self.weekly_results = {}
        for ing_id in ingredient_ids:
            y = ingredient_time_series[ing_id]
            x = list(range(len(y)))
            a, b = self.gradient_descent_linear_regression(x, y)
            total_pred_qty = sum(max(a * (len(dates) - 1 + i) + b, 0) for i in range(1, 8))
            self.weekly_results[ing_id] = total_pred_qty

    def gradient_descent_linear_regression(self, x_list, y_list, lr=0.01, epochs=5000):
        a, b = 0.0, 0.0
        n = len(x_list)
        if n == 0:
            return 0.0, 0.0
        for _ in range(epochs):
            da = sum((a * x + b - y) * x for x, y in zip(x_list, y_list)) * 2 / n
            db = sum((a * x + b - y) for x, y in zip(x_list, y_list)) * 2 / n
            a -= lr * da
            b -= lr * db
        return a, b

    def print_output(self):
        name_map = {i['ingredient_id']: i['name'] for i in self.ingredients_data['ingredients']}
        unit_map = {i['ingredient_id']: i.get('unit', 'unit') for i in self.ingredients_data['ingredients']}
        print("Prediction of total demand for each ingredient in the next week (7 days):")
        for ing_id, qty in sorted(self.weekly_results.items(), key=lambda x: -x[1]):
            print(f"- {name_map.get(ing_id, ing_id)}: {qty:.2f} {unit_map.get(ing_id)}")

# Encryption System

c^i(r)=(b^i(r−1)⊕k(i+r) mod m)⊕d(i,r)

In [5]:
import hashlib
import base64
import random

# 简单 Diffie-Hellman 密钥交换
class KeyExchange:
    def __init__(self, g=5, p=97):
        self.g = g
        self.p = p
        self.private = random.randint(1, 50)
        self.public = pow(g, self.private, p)

    def generate_shared_key(self, other_public):
        shared_secret = pow(other_public, self.private, self.p)
        return hashlib.sha256(str(shared_secret).encode()).digest()  # 32字节密钥

# 手写对称加密（类 AES 概念，3轮简单轮换+替换）
def pseudo_encrypt(data: str, key: bytes, rounds: int = 3) -> str:
    data_bytes = data.encode()
    for r in range(rounds):
        data_bytes = bytes([(b ^ key[(i + r) % len(key)]) ^ ((i + r) * 13 % 256) for i, b in enumerate(data_bytes)])
    return base64.b64encode(data_bytes).decode()

def pseudo_decrypt(enc_data: str, key: bytes, rounds: int = 3) -> str:
    data_bytes = base64.b64decode(enc_data)
    for r in reversed(range(rounds)):
        data_bytes = bytes([(b ^ ((i + r) * 13 % 256)) ^ key[(i + r) % len(key)] for i, b in enumerate(data_bytes)])
    return data_bytes.decode()

# Main system
```python
#Example of data getter
system = system_manager()

for SubClass in (Manager, Cashier, Chef, Customer):
    inst = SubClass()
    decrypted_data = inst.retrieve_data(system)
    decrypted_data = str_to_json(decrypted_data)
    print(f"{SubClass.__name__} decrypted:", decrypted_data)
```

In [6]:
class system_manager:
    def __init__(self):
        data_banker = datasets()
        self.users_available = data_banker.get_users()

        """角色数据映射"""
        self._plain_data = {
            Manager: {
                "user": data_banker.get_users(),
                "order": data_banker.get_orders(),
                "inventory": data_banker.get_iventory(),
                "ingredient_order": data_banker.get_ingredient_order(),
                "finance": data_banker.get_finance(),
                "feedbacks": data_banker.get_feedback(),
                "threshold": data_banker.get_threshold()
            },
            Cashier: {
                "menu": data_banker.get_menu(),
                "order": data_banker.get_orders()
            },
            Chef: {
                "ingr": data_banker.get_iventory()
            },
            Customer: {
                "menu": data_banker.get_menu(),
                "feedback": data_banker.get_feedback()
            }
        }

        # 用于每个角色的密钥（示例：从角色类名哈希派生）
        self.kex = KeyExchange()
        self.shared_key = None

    def exchange_public_key(self):
        return self.kex.public

    def receive_public_key(self, other_pub):
        self.shared_key = self.kex.generate_shared_key(other_pub)

    def send_message(self, message):
        encrypted = pseudo_encrypt(message, self.shared_key)
        return encrypted

    def get_data(self, requester) -> str:
        cls = type(requester)
        if cls not in self._plain_data:
            raise ValueError(f"Unknown requester: {cls}")
        plain = str(self._plain_data[cls])

        if cls is Manager:
            dt = self.send_message(plain)
            return dt
        elif cls is Cashier:
            dt = self.send_message(plain)
            return dt
        elif cls is Chef:
            dt = self.send_message(plain)
            return dt
        elif cls is Customer:
            dt = self.send_message(plain)
            return dt

    """Login Function"""
    def login(self, accID, password):
        for user in self.users_available['users']:
            if user["accID"] == accID and user["pass"] == password:
                return True, user['role']
        return False, None

    """Manager Authorization"""
    def manager_only(method):
        def wrapper(self, requester, *args, **kwargs):
            if requester.__class__.__name__ == 'Manager':
                return method(self, requester, *args, **kwargs)
            print(f"Permission denied: {method.__name__} requires Manager role.")
        return wrapper

    @manager_only
    def add_user(self, requester, user):
        self.users_available['users'].append(user)
        write(self.users_available, "datasets/users.txt")

    @manager_only
    def write_users(self, requester, users_dict):
        write(users_dict, 'datasets/users.txt')

    @manager_only
    def write_orders(self, requester, orders_dict):
        write(orders_dict, 'datasets/orders.txt')

    @manager_only
    def write_ingredients(self, requester, inv_dict):
        write(inv_dict, 'datasets/ingredient_order.txt')

    """Cashier Authorization"""
    def cashier_only(method):
        def wrapper(self, requester, *args, **kwargs):
            if requester.__class__.__name__ == 'Cashier':
                return method(self, requester, *args, **kwargs)
            print(f"Permission denied: {method.__name__} requires Cashier role.")
        return wrapper

    @cashier_only
    def write_menu(self, requester, menu_dict):
        write(menu_dict, 'datasets/menu.txt')

    @cashier_only
    def write_orders(self, requester, order_dict):
        write(order_dict, 'datasets/orders.txt')

# Manager

In [7]:
    def print_all_ingredients(self, threshold=5):
        f_data = self.get_item()
        inv = f_data['inventory']
        print("Ingredients inventory list (items with insufficient inventory are bolded)：")
        for item in inv['ingredients']:
            name = item['name']
            if item['stock'] <= threshold:
                name = f"\033[1m{name}\033[0m"
            print(f"- {item['ingredient_id']} {name}: {item['stock']}{item['unit']}")

In [8]:
class Manager:
    def __init__(self, system, panel):
        self.system = system
        self.panel = panel
        self.kex = KeyExchange()
        self.shared_key = None
        
    """Atbash Decryption Function"""
    def retrieve_data(self):
        encrypted = self.system.get_data(self)
        return encrypted

    def exchange_public_key(self):
        return self.kex.public

    def receive_public_key(self, other_pub):
        self.shared_key = self.kex.generate_shared_key(other_pub)

    def get_item(self):
        encrypted_message = self.retrieve_data()
        decrypted = pseudo_decrypt(encrypted_message, self.shared_key)
        decrypted = str_to_json(decrypted)
        return decrypted

    """System Administration"""
    def approve(self, accID):
        for user in self.panel.pending_users:
            if user['accID'] == accID:
                self.system.add_user(self, user)
                self.panel.pending_users.remove(user)
                print(f"The administrator approved the registration of user: {accID}.")
                return
        print(f"No pending user named {accID} found.")

    def reject(self, accID):
        for user in self.panel.pending_users:
            if user['accID'] == accID:
                self.panel.pending_users.remove(user)
                print(f"The administrator denied registration for user {accID}.")
                return
        print(f"No pending user named {accID} found.")

    def list_approved(self):
        f_data = self.get_item()
        users_available = f_data['user']
        users = users_available.get('users', [])
        print("List of approved users:")
        for user in users:
            print(f"- {user['accID']} (Role: {user['role']})")
        return users

    def modify_user(self, accID, new_role=None, new_password=None):
        f_data = self.get_item()
        users_available = f_data['user']
        for user in users_available['users']:
            if user['accID'] == accID:
                if new_role:
                    user['role'] = new_role
                if new_password:
                    user['pass'] = new_password
                self.system.write_users(self, users_available)
                print(f"User updated: {accID}")
                return
        print(f"User {accID} not found")

    """Order Management"""
    def list_pending_orders(self):
        f_data = self.get_item()
        orders_data = f_data['order']
        pending = [o for o in orders_data['orders'] if o['status'].lower() != 'completed']
        if not pending:
            print("There are currently no outstanding orders.")
        else:
            print("List of uncompleted orders:")
            for o in pending:
                print(f"- OrderID: {o['order_id']}, CustomerID: {o['customer_id']}, Time: {o['order_time']}, Status: {o['status']}, Amount: {o['total']}")
        return pending

    def update_order_status(self, order_id, new_status):
        f_data = self.get_item()
        orders_data = f_data['order']
        for o in orders_data['orders']:
            if o['order_id'] == order_id:
                o['status'] = new_status
                self.system.write_orders(self, orders_data)
                print(f"Updated order {order_id} status to {new_status}")
                return
        print(f"Order {order_id} not found")

    """Financial Management"""
    def financial_summary_past_week(self):
        f_data = self.get_item()
        orders_data = f_data['order']
        all_orders = orders_data.get('orders', [])

        now = datetime.now().date()
        week_ago = now - timedelta(days=7)

        recent_orders = []
        for order in all_orders:
            try:
                order_time = datetime.fromisoformat(order['order_time']).date()
            except Exception as e:
                print(f"Order {order['order_id']} time format error: {e}")
                continue

            if week_ago <= order_time <= now and order['status'] == 'Completed':
                recent_orders.append(order)

        if not recent_orders:
            print("No orders in the past week.")
            return []

        print(f"Order statistics in the past week (a total of {len(recent_orders)} orders):")
        total_amount = 0.0
        for order in recent_orders:
            print(f"- OrderID: {order['order_id']}, CustomerID: {order['customer_id']}, Time: {order['order_time']}, Status: {order['status']}, Amount: {order['total']}")
            total_amount += order.get('total', 0.0)

        print(f"Total amount：{total_amount:.2f}")
        return recent_orders

    def prediction_output(self):
        model = Model()
        print(model.print_output())
        
    """Inventory Control"""
    def print_all_ingredients(self):
        f_data = self.get_item()
        inv = f_data['inventory']
        thresholds = f_data['threshold']
        print("Ingredients inventory list (stock <= threshold 的项目会加粗显示)：")
        for item in inv['ingredients']:
            ing_id = item['ingredient_id']
            name = item['name']
            stock = item['stock']
            unit = item['unit']
            if ing_id in thresholds:
                th = thresholds[ing_id]
                if stock <= th:
                    name = f"\033[1m{name}\033[0m"
            print(f"- {ing_id} {name}: {stock}{unit}")

        
    def add_purchase_order(self, ingredient_id, quantity):
        f_data = self.get_item()
        inv = f_data['inventory']
        ing = next((it for it in inv['ingredients'] if it['ingredient_id'] == ingredient_id), None)
        if not ing:
            print(f"Ingredient with ID {ingredient_id} not found.")
            return
        unit = ing['unit']
        price_per_unit = ing['price_per_unit']
        total_price = round(price_per_unit * quantity, 2)
        po_data = f_data['ingredient_order']
        next_id = (po_data['purchase_orders'][-1]['purchase_id'] + 1) if po_data['purchase_orders'] else 1
        po = {
            'purchase_id': next_id,
            'ingredient_id': ingredient_id,
            'ingredient_name': ing['name'],
            'quantity': f"{quantity}{unit}",
            'total_price': total_price,
            'order_time': datetime.now().isoformat(),
            'status': 'Ordered'
        }
        po_data['purchase_orders'].append(po)
        self.system.write_ingredients(self, po_data)
        print(f"ReplenishmentID {next_id}: Food {ing['name']} (ID:{ingredient_id}), Quantity {quantity}{unit}, Total price {total_price}, Time {po['order_time']}")

    def list_purchase_orders(self):
        f_data = self.get_item()
        po_dict = f_data['ingredient_order']
        if isinstance(po_dict, list):
            orders = po_dict
        else:
            orders = po_dict.get("purchase_orders", [])
        print("Current replenishment order：")
        for po in orders:
            print(f"- ReplenishmentID: {po['purchase_id']}, Food: {po['ingredient_name']} (ID:{po['ingredient_id']}), Quantity: {po['quantity']}, Total price: {po['total_price']}, Time: {po['order_time']}, Status: {po['status']}")
        return orders

    def update_purchase_order_status(self, rid, stat):
        f_data = self.get_item()
        po_dict = f_data['ingredient_order']
        orders = po_dict.get("purchase_orders", [])
        for od in orders:
            if od['purchase_id'] == int(rid):
                od['status'] =  stat
        self.system.write_ingredients(self, po_dict)
        print(f"Has updated ReplenishmentID {rid}'s status to {stat}.")
        
    def delete_purchase_order(self, purchase_id):
        f_data = self.get_item()
        po_dict = f_data['ingredient_order']
        if not isinstance(po_dict, dict) or 'purchase_orders' not in po_dict:
            print("There are no replenishment orders to delete.")
            return
        orders = po_dict['purchase_orders']
        for idx, o in enumerate(orders):
            if o['purchase_id'] == purchase_id:
                orders.pop(idx)
                self.system.write_ingredients(self, po_dict)
                print(f"Purchase order {purchase_id} has been deleted")
                return
        print(f"Purchase order {purchase_id} not found")

    """Customer Feedback"""
    def list_feedback(self):
        print("Feedbacks：")
        f_data = self.get_item()
        fk = f_data['feedbacks']
        for fb in fk.get('feedback', []):
            print(f"- ID:{fb['feedback_id']} OrderID:{fb['order_id']} CustomerID:{fb['customer_id']} DishID:{fb['dish_id']} Rating:{fb['rating']} Comment:{fb['comment']} Time:{fb['time']}")

In [9]:
def manager_interface(mgr):
    while True:
        print("\n[Manager] \nOptions: 1.Users 2.Orders 3.Inventory 4.Financial Summary 5.Feedback 6.Sign Out")
        cmd = input('> ')
        if cmd == '1': 
            print("\n[Manager] Users: \nOptions: 1.Pending approval 2.Approved 3.Quit")
            choice = input('> ')
            if choice == '1':
                mgr.panel.show_pending()
                print("\n[Manager] Users(Pending approval): \nOptions: 1.Approve 2.Reject 3.Quit")
                option = input('> ')
                if option == '1':
                    mgr.approve(input('Enter the accID to approve: '))
                elif option == '2':
                    mgr.reject(input('Enter the accID to reject: '))
                elif option == '3':
                    continue
                else:
                    print("Invalid Option")
            elif choice == '2':
                mgr.list_approved()
                print("\n[Manager] Users(Approved): \nOptions: 1.Modify user 2.Quit")
                choice = input('> ')
                if choice == '1':
                    uid = input('Enter the accID to be modified: ')
                    nr = input('New role (optional): ')
                    np = input('New password(optional): ')
                    mgr.modify_user(uid, new_role=nr or None, new_password=np or None)
                elif choice == '2':
                    continue
                else:
                    print("Invalid Option")
            elif choice == '3':
                continue
            else:
                print("Invalid Option")
        elif cmd == '2': 
            mgr.list_pending_orders()
            print("\n[Manager] Orders: \nOptions: 1.Update order 2.Quit")
            choice = input('> ')
            if choice == '1':
                ID = int(input('OrderID: '))
                State = input('Status (Completed/In progress/Cancelled): ')
                if State == 'Completed' or State == 'In progress' or State == 'Cancelled':
                    mgr.update_order_status(ID, State)
                else:
                    print('Invalid Status')
            elif choice == '2':
                continue
            else:
                print("Invalid Option")
        elif cmd == '3': 
            mgr.print_all_ingredients()
            print("\n[Manager] Inventory: \nOptions: 1.Next Week Inventory Requirements Prediction 2.Quit")
            choice = input('> ')
            
            if choice == '1':
                mgr.prediction_output()
                print("\n[Manager] Prediction Complete: \nOptions: 1.Replenishment 2.Quit")
                sub_choice = input('> ')
                if sub_choice == '1':
                    mgr.list_purchase_orders()
                    print("\n[Manager] Inventory (Replenishment): \nOptions: 1.New Replenishment 2.Update Replenishment 3.Quit")
                    option = input('> ')
                    if option == '1':
                        print("Ingredient list：")
                        inv = load('datasets/ingredients.txt')
                        for it in inv['ingredients']:
                            print(f"- ID: {it['ingredient_id']} Name: {it['name']} Unit: {it['unit']} Price(RM): {it['price_per_unit']}/{it['unit']}")
                        iid = int(input('Ingredient ID: '))
                        qty = float(input('Amount (in unit): '))
                        mgr.add_purchase_order(iid, qty)
                    elif option == '2':
                        pid = int(input('Replenishment ID: '))
                        print('\n[Manager] \nOptions: 1.Update status  2.Delete')
                        state = input('> ')
                        if state == '1':
                            new = input('New status (Ordered, Received, Cancelled): ')
                            if new in ['Ordered', 'Received', 'Cancelled']:
                                mgr.update_purchase_order_status(pid, new)
                            else:
                                print("Invalid status")
                        elif state == '2':
                            mgr.delete_purchase_order(pid)
                        elif state == '3':
                            pass
                        else:
                            print("Invalid Option")
                    elif option == '3':
                        pass
                    else:
                        print("Invalid Option")
                
                elif sub_choice == '2':
                    pass
                else:
                    print("Invalid Option")
            elif choice == '2':
                pass
            else:
                print("Invalid Option")
        elif cmd =='4': 
            mgr.financial_summary_past_week()
            print('\n[Manager] \nOptions: 1.Quit')
            choice = input('> ')
            if choice == '1':
                continue
            else:
                print("Invalid Option")
        elif cmd =='5': 
            mgr.list_feedback()
        elif cmd == '6': 
            break
        else: print("Invalid Option")

# Cashier

In [10]:
# pseudo_decrypt, str_to_json, KeyExchange 等需自己补充
class Cashier():
    def __init__(self, system=None, panel=None):
        self.system = system
        self.panel = panel
        self.kex = KeyExchange()
        self.shared_key = None
        
        self.image_refs = []
        self.total = 0
        self.total_label = None
        self.cart = []
        self.as_tuples = [tuple(item) for item in self.cart]
        self.CRow = 0
        self.label_widgets = {}
        self.CWindow = None
        self.category_frames = {}

    # --- 核心加密通讯相关 ---
    def retrieve_data(self):
        encrypted = self.system.get_data(self)
        return encrypted

    def exchange_public_key(self):
        return self.kex.public

    def receive_public_key(self, other_pub):
        self.shared_key = self.kex.generate_shared_key(other_pub)

    def get_item(self):
        encrypted_message = self.retrieve_data()
        decrypted = pseudo_decrypt(encrypted_message, self.shared_key)
        decrypted = str_to_json(decrypted)
        return decrypted

    # --- 购物车操作 ---
    def add_to_cart(self, CWindow, name, price):
        self.cart.append([name, price])
        name_list = [item[0] for item in self.cart]
        name_counts = Counter(name_list)
    
        name_to_price = {}
        for item_name, item_price in self.cart:
            name_to_price[item_name] = item_price
    
        # 清除旧控件
        for widget in self.label_widgets.values():
            widget.destroy()
        self.label_widgets = {}
    
        # 重新绘制购物车项目
        for row_index, (item_name, count) in enumerate(name_counts.items()):
            item_price = name_to_price[item_name]
            self.label_widgets[(row_index, 0)] = Label(CWindow, text=f"{item_name}", font=("Arial", 14, "bold"))
            self.label_widgets[(row_index, 1)] = Label(CWindow, text=f"{count}", font=("Arial", 14, "bold"))
            self.label_widgets[(row_index, 2)] = Label(CWindow, text=f"{item_price:.2f}", font=("Arial", 14, "bold"))
    
            self.label_widgets[(row_index, 0)].grid(row=row_index, column=0)
            self.label_widgets[(row_index, 1)].grid(row=row_index, column=1)
            self.label_widgets[(row_index, 2)].grid(row=row_index, column=2)
    
        self.total += price
    
        if self.total_label is None:
            self.total_label = Label(CWindow, text=f"Total: {self.total}")
            self.total_label.grid(row=self.CRow + 1, column=0)
        else:
            self.total_label.config(text=f"Total: {self.total}")
        self.CRow += 1

    # --- 打印订单 ---
    def PrintOut(self):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        self.orders = f_data['order']
        
        try:
            name_list = [tuple(item) for item in self.cart]
            counts = Counter(name_list)
    
            totalOrderIDs = [item["order_id"] for item in self.orders["orders"]]
            orderID = max(totalOrderIDs, default=5000) + 1
    
            customerID_num = len(self.orders["orders"]) + 201
            customerID = f"C{customerID_num}"
    
            order_items = []
            total = 0.0
    
            for (name, price), quantity in counts.items():
                dish = next((item for item in self.menu["menu_items"] if item["name"] == name), None)
                if not dish:
                    continue
    
                dish_id = dish["dish_id"]
                order_items.append({
                    "order_id": orderID,
                    "dish_id": dish_id,
                    "quantity": quantity,
                    "unit_price": price
                })
    
                total += quantity * price
    
            self.orders["orders"].append({
                "order_id": orderID,
                "customer_id": customerID,
                "order_time": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
                "status": "Completed",
                "total": round(total, 2)
            })
    
            self.orders["order_items"].extend(order_items)
    
            messagebox.showinfo(message="Order submitted successfully.")
            self.system.write_orders(self, self.orders)
    
        except Exception as e:
            messagebox.showerror(message=f"Order submission failed:\n{e}")

    # --- 增加商品 ---
    def Adding(self, addingTextBox, addingPrice, addingCategory, addingImg):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        try:
            foodName = addingTextBox.get().title()
            foodPrice = float(addingPrice.get())
            foodCategory = addingCategory.get().title()
            foodImg = addingImg.get()
    
            menuIDs = [item["dish_id"] for item in self.menu["menu_items"]]
            foodID = max(menuIDs, default=1000) + 1
    
            self.menu["menu_items"].append({
                "dish_id": foodID,
                "name": foodName,
                "image": foodImg,
                "category": foodCategory,
                "price": foodPrice,
                "available": True
            })
    
            self.system.write_menu(self, self.menu)
            self.refresh_all_tab()
    
            if foodCategory in self.category_frames:
                self.refresh_category_tab(foodCategory)
                messagebox.showinfo(message="Item added successfully!")
            else:
                messagebox.showinfo(message=f"Item added to '{foodCategory}', but tab does not exist (may need restart).")
        except Exception as e:
            messagebox.showerror(message=f"Failed to add item.\nError: {e}")

    # --- 文件选择 ---
    def openFile(self, imagePathVar):
        path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.gif *.bmp")]
        )
        if path:
            imagePathVar.set(path)
            messagebox.showinfo(message="Picture added successfully")

    # --- 删除商品 ---
    def deletingItem(self, DelName, Delcategory):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        try:
            name = DelName.get().title()
            category = Delcategory.get().title()
    
            itemIndex = next(
                i for i, item in enumerate(self.menu["menu_items"])
                if item["name"] == name and item["category"] == category
            )
    
            removed_item = self.menu["menu_items"].pop(itemIndex)
    
            messagebox.showinfo(message=f"Item '{removed_item['name']}' deleted successfully.")
            self.refresh_all_tab()
            self.refresh_category_tab(category)
            self.system.write_menu(self, self.menu)
        except StopIteration:
            messagebox.showerror(message="Item not found. Please check the name and category.")
        except Exception as e:
            messagebox.showerror(message=f"An unexpected error occurred:\n{e}")

    # --- 提价 ---
    def UpdateIncrease(self, UpdateName, Updatecategory, UpdatePercentage):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        try:
            name = UpdateName.get().title()
            category = Updatecategory.get().title()
            percentage = (100 + float(UpdatePercentage.get())) / 100
    
            found = False
            for item in self.menu["menu_items"]:
                if item["name"] == name and item["category"] == category:
                    old_price = item["price"]
                    item["price"] = round(old_price * percentage, 2)
                    found = True
                    break
    
            if found:
                messagebox.showinfo(message=f"Item '{name}' updated successfully.")
                self.refresh_all_tab()
                self.refresh_category_tab(category)
                self.system.write_menu(self, self.menu)
            else:
                messagebox.showerror(message="Item not found. Please check the name and category.")
        except ValueError:
            messagebox.showerror(message="Invalid percentage value.")
        except Exception as e:
            messagebox.showerror(message=f"An unexpected error occurred:\n{e}")

    # --- 降价 ---
    def UpdateDecrease(self, UpdateName, Updatecategory, UpdatePercentage):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        try:
            name = UpdateName.get().title()
            category = Updatecategory.get().title()
            percentage = (100 - float(UpdatePercentage.get())) / 100
    
            found = False
            for item in self.menu["menu_items"]:
                if item["name"] == name and item["category"] == category:
                    old_price = item["price"]
                    item["price"] = round(old_price * percentage, 2)
                    found = True
                    break
    
            if found:
                messagebox.showinfo(message=f"Item '{name}' updated successfully.")
                self.refresh_all_tab()
                self.refresh_category_tab(category)
                self.system.write_menu(self, self.menu)
            else:
                messagebox.showerror(message="Item not found. Please check the name and category.")
        except ValueError:
            messagebox.showerror(message="Invalid percentage value.")
        except Exception as e:
            messagebox.showerror(message=f"An unexpected error occurred:\n{e}")

    # --- 画圆角边框 ---
    def draw_rounded_border(self, event, canvas):
        canvas.delete("border")
    
        w = event.width
        h = event.height
        r = 20          # Corner radius
        margin = 10     # Border margin
        stroke_width = 3
        fill_color = "#fff5ec"
        border_color = "#e4791c"
    
        canvas.create_arc(margin, margin, margin+2*r, margin+2*r,
                          start=90, extent=90, style=PIESLICE,
                          fill=fill_color, outline=fill_color, tags="border")
        canvas.create_arc(w - margin - 2*r, margin, w - margin, margin + 2*r,
                          start=0, extent=90, style=PIESLICE,
                          fill=fill_color, outline=fill_color, tags="border")
        canvas.create_arc(w - margin - 2*r, h - margin - 2*r, w - margin, h - margin,
                          start=270, extent=90, style=PIESLICE,
                          fill=fill_color, outline=fill_color, tags="border")
        canvas.create_arc(margin, h - margin - 2*r, margin + 2*r, h - margin,
                          start=180, extent=90, style=PIESLICE,
                          fill=fill_color, outline=fill_color, tags="border")
    
        canvas.create_rectangle(margin + r, margin, w - margin - r, margin + r,
                                fill=fill_color, outline=fill_color, tags="border")
        canvas.create_rectangle(margin + r, h - margin - r, w - margin - r, h - margin,
                                fill=fill_color, outline=fill_color, tags="border")
        canvas.create_rectangle(margin, margin + r, margin + r, h - margin - r,
                                fill=fill_color, outline=fill_color, tags="border")
        canvas.create_rectangle(w - margin - r, margin + r, w - margin, h - margin - r,
                                fill=fill_color, outline=fill_color, tags="border")
        canvas.create_rectangle(margin + r, margin + r, w - margin - r, h - margin - r,
                                fill=fill_color, outline=fill_color, tags="border")
    
        canvas.create_arc(margin, margin, margin+2*r, margin+2*r,
                          start=90, extent=90, style=ARC,
                          width=stroke_width, outline=border_color, tags="border")
        canvas.create_arc(w - margin - 2*r, margin, w - margin, margin + 2*r,
                          start=0, extent=90, style=ARC,
                          width=stroke_width, outline=border_color, tags="border")
        canvas.create_arc(w - margin - 2*r, h - margin - 2*r, w - margin, h - margin,
                          start=270, extent=90, style=ARC,
                          width=stroke_width, outline=border_color, tags="border")
        canvas.create_arc(margin, h - margin - 2*r, margin + 2*r, h - margin,
                          start=180, extent=90, style=ARC,
                          width=stroke_width, outline=border_color, tags="border")
    
        canvas.create_line(margin + r, margin, w - margin - r, margin,
                           width=stroke_width, fill=border_color, tags="border")
        canvas.create_line(w - margin, margin + r, w - margin, h - margin - r,
                           width=stroke_width, fill=border_color, tags="border")
        canvas.create_line(w - margin - r, h - margin, margin + r, h - margin,
                           width=stroke_width, fill=border_color, tags="border")
        canvas.create_line(margin, h - margin - r, margin, margin + r,
                           width=stroke_width, fill=border_color, tags="border")

    # --- 添加商品UI Tab ---
    def AddItems(self, SettingsNotebook):
        AddTab = Frame(SettingsNotebook)
        SettingsNotebook.add(AddTab, text="ADD ITEMS")
        AddTab.configure(bg="#387647")
    
        canvas = Canvas(AddTab, bg="#387647", highlightthickness=0, bd=0)
        canvas.place(relx=0, rely=0, relwidth=1, height=350)
        canvas.bind("<Configure>", lambda event: self.draw_rounded_border(event, canvas))
    
        addOuter = Frame(AddTab, bg="#fff5ec")
        addOuter.place(relx=0.5, y=150, relwidth=0.9, anchor="center")
    
        addOuter.grid_columnconfigure(0, weight=1)
        addOuter.grid_columnconfigure(1, weight=1)
        addOuter.grid_columnconfigure(2, weight=1)
    
        Label(addOuter, text="ADD ITEM", font=("Showcard Gothic", 20, "bold"), bg="#fff5ec").grid(row=0, column=0, columnspan=3)
    
        Label(addOuter, text="Name", bg="#fff5ec").grid(row=1, column=0)
        addingName = Entry(addOuter, font=("Arial", 25))
        addingName.grid(row=1, column=1, columnspan=2, sticky="ew")
    
        Label(addOuter, text="Price", bg="#fff5ec").grid(row=2, column=0)
        addingPrice = Entry(addOuter, font=("Arial", 25))
        addingPrice.grid(row=2, column=1, columnspan=2, sticky="ew")
    
        Label(addOuter, text="Category", bg="#fff5ec").grid(row=3, column=0)
        addingCategory = Entry(addOuter, font=("Arial", 25))
        addingCategory.grid(row=3, column=1, columnspan=2, sticky="ew")
    
        imagePathVar = StringVar()
        Label(addOuter, text="Image", bg="#fff5ec").grid(row=4, column=0)
        Button(addOuter, text="Insert", command=lambda: self.openFile(imagePathVar)).grid(row=4, column=1, columnspan=2, sticky="ew")
    
        Label(addOuter, text="", bg="#fff5ec").grid(row=5, column=0)
    
        Button(addOuter, text="Add", bg="#f7cf93", font=("Arial", 10), command=lambda: self.Adding(addingName, addingPrice, addingCategory, imagePathVar)).grid(row=6, column=0, columnspan=3)

    # --- 删除商品UI Tab ---
    def DeletingItems(self, SettingsNotebook):
        DelTab = Frame(SettingsNotebook)
        SettingsNotebook.add(DelTab, text="DELETE ITEMS")
        DelTab.configure(bg="#387647")
    
        canvas = Canvas(DelTab, bg="#387647", highlightthickness=0, bd=0)
        canvas.place(relx=0, rely=0, relwidth=1, height=350)
        canvas.bind("<Configure>", lambda event: self.draw_rounded_border(event, canvas))
    
        DelOuter = Frame(DelTab, bg="#fff5ec")
        DelOuter.place(relx=0.5, y=150, relwidth=0.9, anchor="center")
    
        DelOuter.grid_columnconfigure(0, weight=1)
        DelOuter.grid_columnconfigure(1, weight=1)
        DelOuter.grid_columnconfigure(2, weight=1)
    
        Label(DelOuter, text="Delete a dish", font=("Showcard Gothic", 20, "bold"), bg="#fff5ec").grid(row=0, column=0, columnspan=3)
        Label(DelOuter, bg="#fff5ec").grid(row=1, column=0)
    
        Label(DelOuter, text="Name", bg="#fff5ec").grid(row=2, column=0)
        DelName = Entry(DelOuter, font=("Arial", 25))
        DelName.grid(row=2, column=1, columnspan=2, sticky="ew")
    
        Label(DelOuter, text="Category", bg="#fff5ec").grid(row=3, column=0)
        Delcategory = Entry(DelOuter, font=("Arial", 25))
        Delcategory.grid(row=3, column=1, columnspan=2, sticky="ew")
    
        Label(DelOuter, bg="#fff5ec").grid(row=4, column=0)
        Button(DelOuter, text="Delete", bg="#f7cf93", font=("Arial", 10), command=lambda: self.deletingItem(DelName, Delcategory)).grid(row=5, column=0, columnspan=3)

    # --- 更新商品UI Tab ---
    def UpdateItems(self, SettingsNotebook):
        UpdateTab = Frame(SettingsNotebook)
        SettingsNotebook.add(UpdateTab, text="UPDATE ITEMS")
        UpdateTab.configure(bg="#387647")
    
        canvas = Canvas(UpdateTab, bg="#387647", highlightthickness=0, bd=0)
        canvas.place(relx=0, rely=0, relwidth=1, height=350)
        canvas.bind("<Configure>", lambda event: self.draw_rounded_border(event, canvas))
    
        UpdateOuter = Frame(UpdateTab, bg="#fff5ec")
        UpdateOuter.place(relx=0.5, y=150, relwidth=0.9, anchor="center")
    
        UpdateOuter.grid_columnconfigure(0, weight=1)
        UpdateOuter.grid_columnconfigure(1, weight=1)
        UpdateOuter.grid_columnconfigure(2, weight=1)
    
        Label(UpdateOuter, text="Update a dish", font=("Showcard Gothic", 20, "bold"), bg="#fff5ec").grid(row=0, column=0, columnspan=3)
        Label(UpdateOuter, text="", bg="#fff5ec").grid(row=1, column=0)
    
        Label(UpdateOuter, text="Name", bg="#fff5ec").grid(row=2, column=0)
        UpdateName = Entry(UpdateOuter, font=("Arial", 25))
        UpdateName.grid(row=2, column=1, columnspan=2, sticky="ew")
    
        Label(UpdateOuter, text="", bg="#fff5ec").grid(row=3, column=0)
    
        Label(UpdateOuter, text="Category", bg="#fff5ec").grid(row=4, column=0)
        Updatecategory = Entry(UpdateOuter, font=("Arial", 25))
        Updatecategory.grid(row=4, column=1, columnspan=2, sticky="ew")
    
        Label(UpdateOuter, text="", bg="#fff5ec").grid(row=5, column=0)
    
        Label(UpdateOuter, text="Amount (percentage)", bg="#fff5ec").grid(row=6, column=0)
        UpdatePercentage = Entry(UpdateOuter, font=("Arial", 25))
        UpdatePercentage.grid(row=6, column=1, columnspan=2, sticky="ew")
    
        Label(UpdateOuter, text="", bg="#fff5ec").grid(row=7, column=0)
    
        Button(UpdateOuter, text="Increase", font=("Arial", 10), bg="#f7cf93", command=lambda: self.UpdateIncrease(UpdateName, Updatecategory, UpdatePercentage)).grid(row=8, column=0, columnspan=2)
        Button(UpdateOuter, text="Decrease", font=("Arial", 10), bg="#f7cf93", command=lambda: self.UpdateDecrease(UpdateName, Updatecategory, UpdatePercentage)).grid(row=8, column=1, columnspan=2)

    # --- 打印小票UI Tab ---
    def PrintReceipt(self, notebook):
        printTab = ttk.Frame(notebook)
        notebook.add(printTab, text="Complete")
        printOuter = Frame(printTab, bg="green")
        printOuter.pack(expand=True, fill="both")
    
        Button(printOuter, text="Finalize Order", height=20, width=40, command=self.PrintOut).pack()

    # --- 菜单设置Tab ---
    def Settings(self, notebook):
        settingsTab = ttk.Frame(notebook)
        notebook.add(settingsTab, text="Menu Settings")
        settingsOuter = Frame(settingsTab, bg="green")
        settingsOuter.pack(expand=True, fill="both")
    
        SettingsNotebook = ttk.Notebook(settingsOuter)
        SettingsNotebook.pack(expand=True, fill="both")
    
        self.AddItems(SettingsNotebook)
        self.DeletingItems(SettingsNotebook)
        self.UpdateItems(SettingsNotebook)

    # --- 打印菜品按钮 ---
    def printCols(self, CWindow, innerFrame, image_path, dish, price):
        if os.path.exists(image_path):
            try:
                img = PhotoImage(file=image_path)
                self.image_refs.append(img)  # prevent garbage collection
                Button(innerFrame, text=f"{dish}\n${price:.2f}", image=img, compound='top',
                       font=("times new roman", 10, "bold"), bg="#fff5ec", command=lambda: self.add_to_cart(CWindow, dish, price)).pack(fill="both", expand=True, padx=10, pady=(5, 2))
            except Exception as e:
                Button(innerFrame, text="Error loading image", bg="#f0f0f0", command=lambda: self.add_to_cart(CWindow, dish, price)).pack()
        else:
            Button(innerFrame, text="Image not found", bg="#f0f0f0", command=lambda: self.add_to_cart(CWindow, dish, price)).pack()

    # --- 刷新所有分类Tab内容 ---
    def refresh_all_tab(self):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        for widget in self.scrollable_frame.winfo_children():
            widget.destroy()

        cols = 3
        row_index = 0

        menu_by_category = defaultdict(list)
        for item in self.menu["menu_items"]:
            menu_by_category[item["category"]].append((item["name"], item["image"], item["price"]))

        for category, items_list in menu_by_category.items():
            category_label = Label(self.scrollable_frame, text=category, font=("Showcard Gothic", 20, "bold"),
                                   anchor="w", bg="#387647", fg="#fff5ec")
            category_label.grid(row=row_index, column=0, columnspan=cols, sticky="w", pady=(10, 0))
            row_index += 1

            for i, (food, image_path, price) in enumerate(items_list):
                col = i % cols
                sub_row = i // cols

                innerFrame = Frame(self.scrollable_frame, bg="#fff5ec", bd=2, relief=GROOVE)
                innerFrame.grid(row=row_index + sub_row, column=col, padx=5, pady=5, sticky="nsew")

                self.printCols(self.CWindow, innerFrame, image_path, food, price)

            row_index += (len(items_list) + cols - 1) // cols

        for i in range(cols):
            self.scrollable_frame.grid_columnconfigure(i, weight=1)

    # --- 刷新单个分类Tab内容 ---
    def refresh_category_tab(self, category):
        f_data = self.get_item()
        self.menu =  f_data['menu']
        
        frame = self.category_frames[category]

        for widget in frame.winfo_children():
            widget.destroy()

        items_list = [
            (item["name"], item["image"], item["price"])
            for item in self.menu["menu_items"]
            if item["category"] == category
        ]

        subCols = 3
        row_index = 0

        for i, (food, image_path, price) in enumerate(items_list):
            col = i % subCols
            sub_row = i // subCols

            subinnerFrame = Frame(frame, bg="#fff5ec", bd=2, relief=GROOVE)
            subinnerFrame.grid(row=row_index + sub_row, column=col, padx=5, pady=5, sticky="nsew")

            self.printCols(self.CWindow, subinnerFrame, image_path, food, price)

        for i in range(subCols):
            frame.grid_columnconfigure(i, weight=1)

    # --- 滚动条 ---
    def scrollBar(self, my_canvas, tab):
        my_scrollbar = Scrollbar(tab, orient=VERTICAL, command=my_canvas.yview)
        my_scrollbar.pack(side=RIGHT, fill=Y)
        my_canvas.configure(yscrollcommand=my_scrollbar.set)
        my_scrollbar.config(command=my_canvas.yview)

    # --- 报表Tab ---
    def report(self, notebook):
        f_data = self.get_item()
        self.orders = f_data['order']
        
        reportTab = ttk.Frame(notebook)
        notebook.add(reportTab, text="report")
        reportOuter = Frame(reportTab, bg="green")
        reportOuter.pack(expand=True, fill="both")

        reportcanvas = Canvas(reportOuter, bg="#387647", highlightthickness=0)
        reportcanvas.pack(side=LEFT, fill="both", expand=True)
        self.scrollBar(reportcanvas, reportOuter)

        report_scrollable_frame = Frame(reportcanvas, bg="#387647")
        reportcanvas.create_window((0, 0), window=report_scrollable_frame, anchor="nw", tags="frame")
        report_scrollable_frame.bind("<Configure>", lambda e: reportcanvas.configure(scrollregion=reportcanvas.bbox("all")))

        def resize_scrollable_frame(event):
            canvas_width = event.width
            reportcanvas.itemconfig("frame", width=canvas_width)

        reportcanvas.bind("<Configure>", resize_scrollable_frame)

        insideCanvas = Canvas(report_scrollable_frame, width=320, height=320, bg="#387647", highlightthickness=0, bd=0)
        combined_results = {}

        for item in self.orders["order_items"]:
            dish_id = item["dish_id"]
            quantity = item["quantity"]
            unit_price = item["unit_price"]

            if dish_id in combined_results:
                combined_results[dish_id][0] += quantity
            else:
                combined_results[dish_id] = [int(quantity), float(unit_price)]

        totalAmountSold = 0

        for dish_id, (quantity, unit_price) in combined_results.items():
            totalAmountSold += (unit_price * quantity)

        colors = ["#b9752f", "#4a2311", "#d13837", "#339c48", "#566a0e", "#702e09", "#b23a20", "#f89f3f", "#cd752b", "#6b9835"]
        totaldegree = 0
        Label(report_scrollable_frame, text="MOST MONEY MADE", font=("Helvetica", 14, "underline"), bg="#387647", fg="white").grid(row=0, column=0)
        insideCanvas.grid(row=1, column=0, rowspan=7)
        Label(report_scrollable_frame, text="Ranking", font=("Helvetica", 14, "underline"), bg="#387647", fg="white").grid(row=0, column=1, columnspan=12)

        ranking_list = sorted(
            combined_results.items(),
            key=lambda item: item[1][0] * item[1][1],
            reverse=True
        )

        columns_per_row = 3
        cols_per_dish = 4
        num = 1
        for y, (dish_id, (quantity, unit_price)) in enumerate(ranking_list):
            degree = (quantity * unit_price / totalAmountSold) * 360
            color = colors[y % len(colors)]
            bgColor = "#f6d7d9"
            insideCanvas.create_arc(20, 20, 300, 300,
                                    start=totaldegree, extent=degree, style=PIESLICE,
                                    fill=color, outline="black")
            totaldegree += degree

            row = (y // columns_per_row) + 1
            base_col = ((y % columns_per_row) * cols_per_dish) + 1

            Label(report_scrollable_frame, text=f"NO.{num}", font=("Helvetica", 14),
                  bg=bgColor, fg=color).grid(row=row, column=base_col, sticky="nsew")

            Label(report_scrollable_frame, text=f"ID:{dish_id}", font=("Helvetica", 14),
                  bg=bgColor, fg=color).grid(row=row, column=base_col + 1, sticky="nsew")

            Label(report_scrollable_frame, text=f"quantity:{quantity}", font=("Helvetica", 14),
                  bg=bgColor, fg=color).grid(row=row, column=base_col + 2, sticky="nsew")

            Label(report_scrollable_frame, text=f"${quantity * unit_price:.2f}", font=("Helvetica", 14),
                  bg=bgColor, fg=color).grid(row=row, column=base_col + 3, sticky="nsew")

            num += 1

        for i in range(columns_per_row * cols_per_dish):
            if i % 4 in [1, 3]:
                report_scrollable_frame.grid_columnconfigure(i, weight=1)
            else:
                report_scrollable_frame.grid_columnconfigure(i, weight=0)

In [11]:
def cashier_interface(app):
    # app 是 Cashier 类的实例，已包含菜单、方法等
    f_data = app.get_item()
    menu =  f_data['menu']

    app.window = Tk()
    app.notebook = ttk.Notebook(app.window)
    tab = Frame(app.notebook)

    app.my_canvas = Canvas(tab)
    app.my_canvas.pack(side=LEFT, fill="both", expand=True)
    app.scrollBar(app.my_canvas, tab)

    app.CWindow = Toplevel(app.window)

    app.scrollable_frame = Frame(app.my_canvas, bg="#387647")
    app.my_canvas.create_window((0, 0), window=app.scrollable_frame, anchor="nw", tags="frame")
    app.scrollable_frame.bind("<Configure>", lambda e: app.my_canvas.configure(scrollregion=app.my_canvas.bbox("all")))

    def resize_scrollable_frame(event):
        canvas_width = event.width
        app.my_canvas.itemconfig("frame", width=canvas_width)

    app.my_canvas.bind("<Configure>", resize_scrollable_frame)

    app.notebook.add(tab, text="ALL")
    app.notebook.pack(expand=True, fill="both")

    cols = 3
    row_index = 0

    menu_by_category = defaultdict(list)
    for item in menu["menu_items"]:
        menu_by_category[item.get("category", "Unknown")].append(
            (item.get("name", "Unknown"), item.get("image", ""), item.get("price", 0))
        )

    for category, items_list in menu_by_category.items():
        category_label = Label(app.scrollable_frame, text=category, font=("Showcard Gothic", 20, "bold"),
                               anchor="w", bg="#387647", fg="#fff5ec")
        category_label.grid(row=row_index, column=0, columnspan=cols, sticky="w", pady=(10, 0))
        row_index += 1

        for i, (food, image_path, price) in enumerate(items_list):
            col = i % cols
            sub_row = i // cols

            innerFrame = Frame(app.scrollable_frame, bg="#fff5ec", bd=2, relief=GROOVE)
            innerFrame.grid(row=row_index + sub_row, column=col, padx=5, pady=5, sticky="nsew")

            app.printCols(app.CWindow, innerFrame, image_path, food, price)

        row_index += (len(items_list) + cols - 1) // cols

    for i in range(cols):
        app.scrollable_frame.grid_columnconfigure(i, weight=1)

    subCols = 3
    for category, items_list in menu_by_category.items():
        subTab = ttk.Frame(app.notebook)
        app.notebook.add(subTab, text=category)
        subOuter = Frame(subTab, bg="#387647")
        subOuter.pack(expand=True, fill="both")
        app.category_frames[category] = subOuter

        for i, (food, image_path, price) in enumerate(items_list):
            col = i % cols
            sub_row = i // cols

            innerFrame = Frame(subOuter, bg="#fff5ec", bd=2, relief=GROOVE)
            innerFrame.grid(row=row_index + sub_row, column=col, padx=5, pady=5, sticky="nsew")

            app.printCols(app.CWindow, innerFrame, image_path, food, price)

        row_index += (len(items_list) + cols - 1) // cols

        for i in range(subCols):
            subOuter.grid_columnconfigure(i, weight=1)

    app.Settings(app.notebook)
    app.PrintReceipt(app.notebook)
    app.report(app.notebook)

    app.window.mainloop()

# Chef

In [12]:
class Chef:
    """使用简单的异或（XOR）加解密"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.xor_key)

    def decrypt(self, data: str, key: int) -> str:
        chars = []
        for i in range(0, len(data), 2):
            byte = int(data[i:i+2], 16) ^ key
            chars.append(_B_CHR(byte))
        return ''.join(chars)

In [13]:
def chef_interface():
    print("进入厨师界面...")

# Customer

In [14]:
class Customer:
    """使用乘法（Mul）加解密——每个字符的码点乘以常数"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.mul_key)

    def decrypt(self, data: str, key: int) -> str:
        chars = []
        for part in data.split(','):
            num = int(part) // key
            chars.append(_B_CHR(num))
        return ''.join(chars)

In [15]:
def customer_interface():
    print("进入顾客界面...")

# Main Panel

In [16]:
class panel:
    def __init__(self):
        self.pending_users = []

    def sign_up(self, acc_name, acc_role, acc_password):
        user = {"accID": acc_name, "pass": acc_password, "role": acc_role}
        self.pending_users.append(user)
        print(f"User {acc_name}({acc_role}) has submitted a registration application and is awaiting approval by the administrator.")

    def show_pending(self):
        if not self.pending_users:
            print("There are no pending users.")
            return []
        print("List of users pending approval：")
        for u in self.pending_users:
            print(f"- {u['accID']} (Role: {u['role']})")
        return self.pending_users

In [17]:
system = system_manager()
pnl = panel()

while True:
    print("\nMain Menu: 1.Sign Up  2.Login  3.Exit")
    choice = input('> ')
    if choice == '1':
        acc = input('Account Name: ')
        rol = input('Account Role(Customer/Chef/Cashier): ')
        pas = input('Account Password: ')
        if rol.lower() == 'customer':
            system.add_user({"accID": acc, "pass": pas, "role": 'customer'})
            print("Customer account created.")
        elif rol.lower() == 'chef' or rol.lower() == 'cashier':
            pnl.sign_up(acc, rol.lower(), pas)
        else:
            print("Invalid role")
    
    elif choice == '2':
        acc = input('Account Name: ')
        pas = input('Account Password: ')
        ok, role = system.login(acc, pas)
        if not ok:
            print("Login failed: wrong account or password.")
            continue
        print(f"Login successful, role: {role.capitalize()}")
        if role == 'manager':
            sys = system_manager()
    
            # 2. 再创建 Manager，并传入同一个 system 实例
            mgr = Manager(sys, pnl)
            
            # 3. 交换密钥
            sys_pub = sys.exchange_public_key()
            mgr_pub = mgr.exchange_public_key()
            
            mgr.receive_public_key(sys_pub)
            sys.receive_public_key(mgr_pub)
            
            # 4. 启动接口
            manager_interface(mgr)
        elif role.lower() == 'cashier':
            sys = system_manager()
            
            chs = Cashier(sys, pnl)

            sys_pub = sys.exchange_public_key()
            chs_pub = chs.exchange_public_key()

            sys.receive_public_key(chs_pub)
            chs.receive_public_key(sys_pub)

            cashier_interface(chs)
        elif role.lower() == 'chef':
            chef_interface()
        elif role.lower() == 'customer':
            app = Cashier()
            customer_interface(app)
        else:
            print("Unknown role, unable to enter the interface.")
    
    elif choice == '3':
        print("Exiting the system.")
        break
        
    else:
        print("Invalid option, please try again. ")


Main Menu: 1.Sign Up  2.Login  3.Exit


>  2
Account Name:  Try
Account Password:  Try


Login successful, role: Cashier

Main Menu: 1.Sign Up  2.Login  3.Exit


>  2
Account Name:  Admin
Account Password:  Admin


Login successful, role: Manager

[Manager] 
Options: 1.Users 2.Orders 3.Inventory 4.Financial Summary 5.Feedback 6.Sign Out


...


[Manager] 
Options: 1.Users 2.Orders 3.Inventory 4.Financial Summary 5.Feedback 6.Sign Out


>  6



Main Menu: 1.Sign Up  2.Login  3.Exit


>  3


Exiting the system.
